# Bimetallic Nanocluster Cohesive Energy Minimization - via Labeling

This notebook serves as an example application of the MatOpt framework. We consider an example optimization problem of identifying the global energy minimum bimetallic nanocluster configuration.

This is a continuation of the example given in ***Monometallic_Nanocluster_Design.ipynb***. In this example, we will show how a very similar model can be used to optimize a bimetallic cluster by "labelling" the sites of a pre-defined monometallic cluster. 

The model for cohesive energy is based on:

Yan, Z., Taylor, M. G., Mascareno, A., & Mpourmpakis, G. (2018). Size-, Shape-, and Composition-Dependent Model for Metal Nanoparticle Stability Prediction. *Nano Letters*, 18(4), 2696-2704.

## Importing Packages

We start by importing several standard Python modules for convienience.

In [1]:
import os 
from math import sqrt

Then, we import the MatOpt package in its entirety. 

In [2]:
from matopt import *

## Setting Up a Material System

We first identify the optimal metal-independent nanocluster shape, using the code that wsas demonstrated in **Monometallic_Nanocluster_Design.ipynb**.

In [3]:
Lat = FCCLattice(IAD=1.0)
Canv = Canvas()
Canv.addLocation(np.array([0,0,0],dtype=float))
Canv.addShells(2,Lat.getNeighbors)
Atoms = [Atom('Cu')]
N = 20
m = MatOptModel(Canv,Atoms)
Vals = [sqrt(CN) for CN in range(0,13)]
BPs = [CN for CN in range(0,13)]
m.addSitesDescriptor('CNRi',bounds=(0,sqrt(12)),integer=False,
                     rules=PiecewiseLinear(values=Vals,
                                           breakpoints=BPs,
                                           input_desc=m.Ci))
m.addGlobalDescriptor('Ecoh',rules=EqualTo(SumSites(desc=m.CNRi,
                                                    coefs=(1/(N*sqrt(12))))))
m.addGlobalDescriptor('Size',bounds=(N,N),
                      rules=EqualTo(SumSites(desc=m.Yi)))
D = m.maximize(m.Ecoh,tilim=100)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.6.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2014.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/tmp/tmpcvroj97s.cplex.log' open.
CPLEX> New value for absolute mixed integer optimality gap tolerance: 0
CPLEX> New value for mixed integer optimality gap tolerance: 0
CPLEX> New value for time limit in seconds: 100
CPLEX> Problem '/tmp/tmpwjv0wzzl.pyomo.lp' read.
Read time = 0.00 sec. (0.18 ticks)
CPLEX> Problem name         : /tmp/tmpwjv0wzzl.pyomo.lp
Objective sense      : Maximize
Variables            :    1315  [Nneg: 716,  Fix: 1,  Box: 55,  Free: 1,
                                 Binary: 487,  General Integer: 55]
Objective nonzeros   :       1
Linear constraints   :    1519  [Less: 

We take the locations from the optimal monometallic problem to initialize a ***Canvas*** object for the bimetallic case. 

In [4]:
Canv = Canvas()
for i in range(len(D)):
    if(D.Contents[i] is not None):
        Canv.addLocation(D.Canvas.Points[i])
Canv.setNeighborsFromFunc(Lat.getNeighbors)

Additionally, we create a few data structures for holding bimetallic material information. First, we make a list of multiple ***Atom*** objects that will be the building blocks of the model. Next, we specify a dictionary with the bounds to impose on composition.

In [5]:
Atoms = [Atom('Cu'),Atom('Ag')]
CompBounds = {Atom('Cu'):(6,6),
              Atom('Ag'):(14,14)}

## Specifying an Optimization Model

We start by creating a ***MatOptModel*** object that will hold the information about the problem variables and constraints. At a minimum, ever model requires a Canvas object to be defined. Additionally, the list of building blocks and conformations that are present in the model should be defined. 

In [6]:
m = MatOptModel(Canv,Atoms)

By default, several basic variables are pre-defined. See the first example, ***Monometallic_Nanocluster_Design.ipynb*** for a description of basic variables, expressions, and constraint rules.

To start, we inidcate that the choice to place an atom is fixed so that each canvas site is required to have an atom. This simplifies the problem significantly and results in a model that will seek to find the optimal labeling of metals on the nanocluster. 

In [7]:
m.Yi.rules.append(FixedTo(1.0))

Next, we define a descriptor for the energy of bonds as a function of properties at each site.
Since the locations of the atoms are fixed, the only decision is how to label each site as either Atom A or Atom B. 
This allows us to simplify the model and compute coefficients that rely on coordination number.
In the block below, we implement the bimetallic model for bond energy defined in Yan et al., 2018.

In [8]:
GklCoefs = {(Atom('Cu'),Atom('Cu')):3.520,
            (Atom('Cu'),Atom('Ag')):2.112,
            (Atom('Ag'),Atom('Ag')):2.580,
            (Atom('Ag'),Atom('Cu')):3.612}
BEijCoefs = {}
for i in range(len(Canv)):
    CNi = sum(1 for _ in Canv.NeighborhoodIndexes[i] if _ is not None)
    for j in Canv.NeighborhoodIndexes[i]:
        if(j is not None):
            CNj = sum(1 for _ in Canv.NeighborhoodIndexes[j] if _ is not None)
            for k in Atoms:
                for l in Atoms:
                    BEijCoefs[i,j,k,l] = GklCoefs[k,l]*1/sqrt(CNi) + GklCoefs[l,k]*1/sqrt(CNj)
m.addBondsDescriptor('BEij',
                     rules=EqualTo(SumBondTypes(m.Xijkl,coefs=BEijCoefs)),
                     symmetric_bonds=True)

Next, we define the cohesive energy as a sum of contributions from all BEij bond descriptors.

In [9]:
m.addGlobalDescriptor('Ecoh',rules=EqualTo(SumBonds(desc=m.BEij,
                                                    coefs=1/(N*sqrt(12)))))

Finally, we add constraints on the size and composition of the resulting designs.

In [10]:
m.addGlobalTypesDescriptor('Composition',bounds=CompBounds,
                           rules=EqualTo(SumSites(desc=m.Yik)))

## Solving the Model

Once the model is fully specified, we can optimize in light of a global descriptor. In this example, we choose to maximize the cohesive energy defined previously. Additionally, we can specify basic optimization parameters such as the time limit and memory limit\* for the optimizer. 


In [11]:
D = m.maximize(m.Ecoh,tilim=360,trelim=4096)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.6.1.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2014.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/tmp/tmp2mk8tqz0.cplex.log' open.
CPLEX> New value for absolute mixed integer optimality gap tolerance: 0
CPLEX> New value for mixed integer optimality gap tolerance: 0
CPLEX> New value for time limit in seconds: 360
CPLEX> New value for upper limit on size of tree in megabytes: 4096
CPLEX> Problem '/tmp/tmpbyvd2xuk.pyomo.lp' read.
Read time = 0.00 sec. (0.07 ticks)
CPLEX> Problem name         : /tmp/tmpbyvd2xuk.pyomo.lp
Objective sense      : Maximize
Variables            :     364  [Nneg: 1,  Fix: 2,  Free: 65,  Binary: 296]
Objective nonzeros   :       1
Linear constraints   :     876  [Les

## Processing Solutions

If a design was identified (optimal or otherwise), then a ***Design*** object is returned from the optimization method. The optimal design can be plotted via any of the supported parsers. 

In [12]:
if(D is not None):
    D.toPDB('result.pdb')